In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/dasarpai/DAI-Datasets/main/amazon_product_review_labelled.txt"
# path=r"D:\github\DL-Amazon-Cells"

df=pd.read_csv(url,names=['sentence', 'label'], sep='\t')
df.head()

,sentence,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [6]:
from sklearn.model_selection import train_test_split             
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences


sentences = df['sentence'].values
y = df['label'].values


sentences_train,sentences_test,y_train,y_test = train_test_split(
                                                sentences, y,  
                                                test_size=0.25,  
                                                random_state=1000)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1                          

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [26]:
#tokenizer.word_index

In [7]:
sentences_train.shape

(750,)

In [8]:
print(X_train.shape)
X_train

(750, 100)


array([[  7,  24,   5, ...,   0,   0,   0],
       [  2,  17, 446, ...,   0,   0,   0],
       [ 17,  13,  86, ...,   0,   0,   0],
       ...,
       [127,  19,  10, ...,   0,   0,   0],
       [ 11,   1,  44, ...,   0,   0,   0],
       [552, 208, 263, ...,   0,   0,   0]])

In [3]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="latin-1") as f:

        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                                        vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [9]:
type(embedding_matrix)
embedding_matrix.shape

(1574, 50)

In [23]:
import itertools
dict(itertools.islice(tokenizer.word_index.items(), 1))

{'the': 1}

In [5]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix("glove.6B.50d.txt",tokenizer.word_index,embedding_dim)

In [29]:
X_train[:1]

array([[  7,  24,   5,  16,   4, 137, 148,   6, 223, 315,   2,  71, 224,
          8,   1, 673, 111, 444,  18, 316,  11, 445,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [32]:
from keras.models import Sequential

from keras import layers
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test),
                    batch_size=10)

D:\Users\admin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 750 samples, validate on 250 samples
Epoch 1/10
750/750 [==============================] - 2s 3ms/step - loss: 0.6871 - accuracy: 0.5453 - val_loss: 0.6786 - val_accuracy: 0.5520
Epoch 2/10
750/750 [==============================] - 2s 3ms/step - loss: 0.5163 - accuracy: 0.7707 - val_loss: 0.6274 - val_accuracy: 0.6920
Epoch 3/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1669 - accuracy: 0.9480 - val_loss: 0.6703 - val_accuracy: 0.7560
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.0323 - accuracy: 0.9947 - val_loss: 0.7662 - val_accuracy: 0.7560
Epoch 5/10
750/750 [==============================] - 2s 3ms/step - loss: 0.0179 - accuracy: 0.9973 - val_loss: 0.8292 - val_accuracy: 0.7720
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 0.0094 - accuracy: 0.9973 - val_loss: 0.9234 - val_accuracy: 0.7480
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.0018 - accuracy: 1.0000 - 